In [1]:
!pip install findspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import findspark

In [3]:
!ls /workspace/code/postgresql-*

/workspace/code/postgresql-42.1.4.jar  /workspace/code/postgresql-42.5.1.jar


In [5]:
findspark.add_jars('/workspace/code/postgresql-42.1.4.jar')
#findspark.add_jars('/workspace/code/postgresql-42.5.1.jar')
findspark.init()

In [6]:
spark = (
    SparkSession.builder
    .appName("read-postgres")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

23/01/08 14:56:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
pas="postgres"

In [9]:
#sql
game_clicks = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/bigdata") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "public.game_clicks") \
    .option("user", "postgres") \
    .option("password", pas) \
    .load()

In [11]:
game_clicks.show(5)

+-------------------+--------+-------+---------------+------+-------+----------+
|          timestamp|click_id|user_id|user_session_id|is_hit|team_id|team_level|
+-------------------+--------+-------+---------------+------+-------+----------+
|1970-01-01 15:06:55|     105|   1038|           5916|     0|     25|         1|
|1970-01-01 15:07:09|     154|   1099|           5898|     0|     44|         1|
|1970-01-01 15:07:14|     229|    899|           5757|     0|     71|         1|
|1970-01-01 15:07:14|     322|   2197|           5854|     0|     99|         1|
|1970-01-01 15:07:20|      22|   1362|           5739|     0|     13|         1|
+-------------------+--------+-------+---------------+------+-------+----------+
only showing top 5 rows



In [12]:
game_clicks.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- click_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session_id: integer (nullable = true)
 |-- is_hit: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_level: integer (nullable = true)



In [13]:
game_clicks.count()

755806

In [14]:
game_clicks.select("user_id", "team_level").show(5)

+-------+----------+
|user_id|team_level|
+-------+----------+
|   1038|         1|
|   1099|         1|
|    899|         1|
|   2197|         1|
|   1362|         1|
+-------+----------+
only showing top 5 rows



In [17]:
game_clicks.filter(game_clicks["team_level"] > 1).select("user_id", "team_level").show(5)

+-------+----------+
|user_id|team_level|
+-------+----------+
|   1513|         2|
|    868|         2|
|   1453|         2|
|   1282|         2|
|   1473|         2|
+-------+----------+
only showing top 5 rows



In [19]:
game_clicks.groupBy("is_hit").count().show()

+------+------+
|is_hit| count|
+------+------+
|     0|672423|
|     1| 83383|
+------+------+



In [20]:
game_clicks.select(mean("is_hit"), sum("is_hit")).show()

+------------------+-----------+
|       avg(is_hit)|sum(is_hit)|
+------------------+-----------+
|0.1103232840173272|      83383|
+------------------+-----------+



In [23]:
#sql
ad_clicks = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/bigdata") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "public.ad_clicks") \
    .option("user", "postgres") \
    .option("password", pas) \
    .load()

In [24]:
ad_clicks.show(3)

+-------------------+-----+---------------+-------+-------+-----+-----------+
|          timestamp|tx_id|user_session_id|team_id|user_id|ad_id|ad_category|
+-------------------+-----+---------------+-------+-------+-----+-----------+
|1970-01-01 15:13:22| 5974|           5809|     27|    611|    2|electronics|
|1970-01-01 15:17:24| 5976|           5705|     18|   1874|   21|     movies|
|1970-01-01 15:22:52| 5978|           5791|     53|   2139|   25|  computers|
+-------------------+-----+---------------+-------+-------+-----+-----------+
only showing top 3 rows



In [25]:
ad_clicks.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- tx_id: integer (nullable = true)
 |-- user_session_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- ad_id: integer (nullable = true)
 |-- ad_category: string (nullable = true)



In [26]:
join = game_clicks.join(ad_clicks, "user_id")

In [27]:
join.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- click_id: integer (nullable = true)
 |-- user_session_id: integer (nullable = true)
 |-- is_hit: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_level: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- tx_id: integer (nullable = true)
 |-- user_session_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- ad_id: integer (nullable = true)
 |-- ad_category: string (nullable = true)



In [29]:
join.show(5)

+-------+-------------------+--------+---------------+------+-------+----------+-------------------+-----+---------------+-------+-----+-----------+
|user_id|          timestamp|click_id|user_session_id|is_hit|team_id|team_level|          timestamp|tx_id|user_session_id|team_id|ad_id|ad_category|
+-------+-------------------+--------+---------------+------+-------+----------+-------------------+-----+---------------+-------+-----+-----------+
|     10|1970-01-01 11:42:18|  206846|          16583|     1|    131|         1|1970-01-01 14:43:05|18179|          16583|    131|    3|electronics|
|     10|1970-01-01 11:42:18|  206846|          16583|     1|    131|         1|1970-01-01 03:24:01|18860|          18403|    131|    4|      games|
|     10|1970-01-01 11:42:18|  206846|          16583|     1|    131|         1|1970-01-01 00:16:48|20153|          18403|    131|   27|      games|
|     10|1970-01-01 11:42:18|  206846|          16583|     1|    131|         1|1970-01-01 05:31:41|20457|